In [11]:
import requests
import json
import pymysql
from newspaper import Article
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'mysql'

In [37]:
# MariaDB Connect
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='root', db='trackle', charset='utf8',autocommit=True)
cursor = db.cursor()

# News 크롤링
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows; U; MSIE 9.0; WIndows NT 9.0; ko-KR))',
}

req = requests.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=", headers=headers)

html = req.text
soup = BeautifulSoup(html, "lxml")

lst_realtime_src = soup.find('ol', {"class":"lst_realtime_srch _tab_area"})
lis = lst_realtime_src.findAll('li')

articles = {"article":[]}

for li in lis:
#   News Url
    key = li.find("span", {"class":"tit"})
    news_page_url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query=" + key.text
    req = requests.get(news_page_url, headers=headers)
    html = req.text
    soup = BeautifulSoup(html, "lxml")
    
    news_list = soup.find("ul", {"class":"type01"}).findAll('li')[0]
    source = news_list.find("span", {"class":"_sp_each_source"})
    link = news_list.find("a")["href"]
    
    req = requests.get(link, headers=headers)
    html = req.text

    a = Article(link, language="ko")
    a.download()
    a.parse()

    title = a.title
    content = a.text
    image = a.top_image
    company = source.text
    keyword = key.text
    
    cursor.execute("INSERT INTO article (title, content, image, company, keyword) VALUES(%s, %s, %s, %s, %s)", (title, content, image, company, keyword))
    db.commit()
db.close()

[]
[]
[]
[]
['부산 여중생 폭행 가해자 추가 구속', '도망할 염려 있어']
[]
[]
['괴짜 노벨상', '걸을 때 커피 쏟는 이유 규명한 한국인 수상']
[]
[]
